In [80]:
# Bibliotecas
import numpy as np
import math



In [81]:
u = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
     0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
v = [4.74, 5.12, 5.39, 5.49, 5.43, 5.21, 4.85,
     4.42, 3.97, 3.56, 3.26, 3.11, 3.13, 3.31, 3.63]


def f(x):
    """Funición a evaluar"""
    a = 0
    for i in range(0, 15):
        aux = x[0]*math.sin(x[1]*u[i])+x[2]-v[i]
        a += aux**2
    return a


def jHess(x):
    jH = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        jH += np.outer(j, j)
    return jH


def jGrad(x):
    jG = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        jG += j*fxerror(x, i)
    return jG

def fxerror(x, i):
    return x[0]*(math.sin(x[1]*u[i]))+x[2]-v[i]


def jacobian(x, i):
    return np.array([math.sin(x[1]*u[i]),
                     u[i]*x[0]*math.cos(x[1]*u[i]),
                     1
                     ])


In [82]:
# gauss newton

def gaussNewton(x0, tol=1e-4):
    k = 0
    print(k, x0)
    while np.linalg.norm(jGrad(x0)) > tol:
        a = jHess(x0)
        s =  np.dot(np.linalg.inv(a), -jGrad(x0))
        x0 = x0 + s
        k = k+1
        print(k,  x0, f(x0))
    return x0

     

In [83]:
#Evaluar Gauss Newton
x0 = np.array([1.0, 2.0, 1.3])
# grad = jHess(x0)
gn = gaussNewton(x0)


0 [1.  2.  1.3]
1 [-0.93568029  3.49735647  5.39809436] 53.37613144510951
2 [1.14538958 3.08262573 4.32987585] 3.2740392361556117
3 [0.8072063  3.95417301 4.54756344] 2.3393236847142225
4 [1.18849459 3.72984109 4.30197552] 0.03666322402337587
5 [1.19729029 3.80013066 4.29694345] 0.0002045916233588216
6 [1.19974186 3.79842054 4.29544624] 0.00010264046529553048


In [86]:
def jHessFull(x):
    jH = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        t = tk(x, i)
        fxe = fxerror(x, i)
        jH += np.outer(j, j) + np.dot(t, fxe)
    return jH

def tk(x, i):
    return np.array([
            [0, u[i]*math.cos(x[1]*u[i]), 0],
            [u[i]*math.cos(x[1]*u[i]), -1*x[0]*(u[i]**2)*math.sin(x[1]*u[i]), 0],
            [0, 0, 0]
        ])


def levenbergMarquardtMC(x0, tol=1e-6):
    k = 0
    mu = 1e-4
    v = 10
    I = np.identity(x0.size)
    print(k, x0, f(x0))
    while np.linalg.norm(jGrad(x0)) >= tol:
        mu = mu/v
        while True:
            delta = np.dot(np.linalg.inv(jHessFull(x0) + mu*I), jGrad(x0))
            x1 = x0 - delta
            mu = mu * v
            # k = k+1
            print(k,  x0, round(f(x0), 6), x1, round(f(x1), 6), mu)
            if (f(x1) < f(x0)):
                x0 = x1
                break
        k = k+1
    return x0


In [87]:
x0 = np.array([1.0, 2.0, 1.3])
# jHessFull(x0)
levenbergMarquardtMC(x0)



0 [1.  2.  1.3] 94.15754849549766
0 [1.  2.  1.3] 94.157548 [3.9958381  1.31259303 1.43304664] 54.018163 0.0001
1 [3.9958381  1.31259303 1.43304664] 54.018163 [-1.30519009  1.55748111  5.12560102] 8.564398 0.0001
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [ 9.87154106  4.9369059  -3.23884731] 1276.464561 0.0001
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [ 9.8788072   4.93941382 -3.24435313] 1278.057288 0.001
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [ 9.95199853  4.9646756  -3.29981285] 1294.155738 0.01
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [10.74108477  5.23699617 -3.89773077] 1471.827327 0.1
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [ 39.89003487  15.28615894 -25.98445334] 25190.161392 1.0
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [-2.9190204   0.56814877  6.4507604 ] 15.336784 10.0
2 [-1.30519009  1.55748111  5.12560102] 8.564398 [-1.37034038  1.28538758  5.25763691] 6.014707 100.0
3 [-1.37034038  1.28538758  5.25763691] 6.014707 [-1.47122

KeyboardInterrupt: 